In [ ]:
# Gerekli kütüphaneleri yükleyelim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.kernel_ridge import KernelRidge
from joblib import dump
import warnings
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore")

# Veri setini yükleme
csv_path = "veri.csv"  # veri.csv ile aynı dizinde çalıştırılmalı
data = pd.read_csv(csv_path)

# Özellikler ve hedef değişken
X = data.drop("Fiyat (TL)", axis=1)
y = data["Fiyat (TL)"]

# Kategorik ve sayısal değişkenlerin ayrılması
categorical_features = ["Cam Türü", "Kaplama Türü", "Marka"]
numerical_features = ["UV Koruma (%)", "Cam Numarası"]

# Ön işleme adımları
categorical_transformer = OneHotEncoder(drop="first", handle_unknown="ignore")
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
        ("num", numerical_transformer, numerical_features),
    ]
)

# Veri setini eğitim ve test kümelerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Algoritmaların tanımlanması
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "Support Vector Regressor": SVR(),
    "K-Nearest Neighbors": KNeighborsRegressor(),
    "Neural Network": MLPRegressor(random_state=42, max_iter=500),
    "Gaussian Process": GaussianProcessRegressor(),
    "Kernel Ridge": KernelRidge(),
}

# Model eğitimi ve değerlendirme
results = {}
for name, model in models.items():
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", model)])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    results[name] = rmse

# Sonuçların sıralanması
sorted_results = sorted(results.items(), key=lambda x: x[1])
best_model_name, best_rmse = sorted_results[0]

# En iyi modeli kaydetme
best_model = models[best_model_name]
final_pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("model", best_model)])
final_pipeline.fit(X_train, y_train)
joblib_path = "eniyi.joblib"
dump(final_pipeline, joblib_path)

# Sonuçları yazdırma
print("RMSE Değerlerine Göre Sıralama:")
for model_name, rmse in sorted_results:
    print(f"{model_name}: {rmse:.2f}")

print(f"\nEn İyi Model: {best_model_name} (RMSE: {best_rmse:.2f})")
print(f"Model kaydedildi: {joblib_path}")
